In [18]:
import pandas as pd, math, time, json

choices = pd.read_csv(
    "2019_choices.csv",
    index_col="code"
)

votes = pd.read_csv(
    "2019_votes.csv",
    usecols=[1, 2],
    index_col=["voter id"]
)
votes = votes.sample(frac=1)
total_votes = len(votes)
quota = math.ceil(total_votes / 3) + 1

for col in ["first", "ALP", "LP", "GRN", "exhausted"]:
    choices.loc[:, col] = 0

def sort_prefs(prefs):
    new_prefs = []    
    for x in prefs:
        if choices.at[x, "btl"]:
            new_prefs.append(x)
        else:
            new_prefs.append(choices[(choices["candidateName"] == choices.at[x, "candidateName"]) & (choices["btl"])].index[0])            
    return new_prefs

def three_party_preferred(prefs):
    prefs_3pp = "".join([x for x in prefs if x in "HJT"])
    if len(prefs_3pp) == 0:
        return [0, 0, 0, 1]
    first_prefs_3pp = prefs_3pp[0]
    if first_prefs_3pp == "T":
        return [1, 0, 0, 0]
    elif first_prefs_3pp == "H":
        return [0, 1, 0, 0]
    else:
        return [0, 0, 1, 0]

choices_3pp = choices.copy()

start = time.time()
interval = 2
print("analysing votes ...", end="\r")
for i, voter in enumerate(votes.index):
    prefs = sort_prefs(votes.at[voter, "prefs"])
    first = prefs[0]
    dist = three_party_preferred(prefs)
    choices_3pp.at[first, "first"] = choices_3pp.at[first, "first"] + 1
    choices_3pp.loc[first, "ALP":"exhausted"] = choices_3pp.loc[first, "ALP":"exhausted"] + dist
    if time.time() - interval > start:
        interval = interval + 2
        print(f"analysing votes ... {(i + 1) / total_votes:.1%}", end="\r")
print("analysing votes ... complete")
choices_3pp.to_csv("2019_results_3pp.csv")

choices_3pp_pc = choices_3pp.copy()
choices_3pp_pc.loc[:, "ALP":"exhausted"] = choices_3pp_pc.loc[:, "ALP":"exhausted"] / total_votes
choices_3pp_pc.to_csv("2019_results_3pp_pc.csv")

choices_3pp_pc.loc[:, "ALP":"exhausted"].sum()

analysing votes ... complete


ALP          0.431657
LP           0.355429
GRN          0.210701
exhausted    0.002213
dtype: float64

In [20]:
choices_3pp

,PartyNm,PartyAb,btl,candidateName,first,ALP,LP,GRN,exhausted
code,,,,,,,,,
A,Liberal,LP,False,SESELJA Zed,0,0,0,0,0
B,The Greens,GRN,False,KYBURZ Penny,0,0,0,0,0
C,Anthony Pesec,Anthony Pesec,False,PESEC Anthony,0,0,0,0,0
D,FRASER ANNING'S CONSERVATIVE NATIONAL PARTY,FACN,False,VAN DUREN Shane,0,0,0,0,0
E,United Australia Party,UAPP,False,WALTER Peter,0,0,0,0,0
F,Sustainable Australia,SPP,False,HAYDON John,0,0,0,0,0
G,Australian Labor Party,ALP,False,GALLAGHER Katy,0,0,0,0,0
H,Liberal,LP,True,SESELJA Zed,84846,0,84846,0,0
I,Liberal,LP,True,GUNNING Robert,2650,669,1567,361,53
